In [111]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras import datasets, layers, models

import pathlib

In [112]:
dataPath = '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/'
dataRoot = pathlib.Path(dataPath)

In [113]:
imageDirs = []
for item in data_root.iterdir():
    imageDirs.append(item)
dataRoot = []
for path in imageDirs:
    dataRoot.append(pathlib.Path(path))

In [114]:
import random

In [115]:
imageLabels = ["Uninfected","Parasitized"]
labeledImages = []
type = 0
for path in dataRoot:
    for image in path.iterdir():
        labeledImages.append((type,str(image)))
    type += 1
random.shuffle(labeledImages)

In [116]:
def loadAndPreprocessImage(path):
  image = tf.read_file(path)
  return preprocessImage(image)

def preprocessImage(image):
  image = tf.image.decode_png(image, channels=3)
  image = tf.image.resize_images(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range

  return image

In [117]:
paths = [x[1] for x in labeledImages]
labels = [x[0] for x in labeledImages]

In [118]:
pathDS = tf.data.Dataset.from_tensor_slices(paths)
imageDS = pathDS.map(loadAndPreprocessImage, num_parallel_calls=tf.data.experimental.AUTOTUNE)
labelDS = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))
imageLabelDS = tf.data.Dataset.zip((imageDS, labelDS))

In [120]:
dataSet = imageLabelDS.shuffle(buffer_size=len(labeledImages))
dataSet = dataSet.repeat()
dataSet = dataSet.batch(512)
dataSet = dataSet.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
dataSet

<DatasetV1Adapter shapes: ((?, 192, 192, 3), (?,)), types: (tf.float32, tf.int64)>

In [ ]:
#LEFT OFF AT PIPE THE DATASET TO A MODEL
#NEED TO:
#    SEPERATE DATA INTO TRAINING AND TESTING
#    TRAIN MODEL
#    TEST MODEL

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0


11493376/11490434 [==============================] - 0s 0us/step


In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 29s 481us/sample - loss: 0.1488 - acc: 0.9535
Epoch 2/5
60000/60000 [==============================] - 28s 460us/sample - loss: 0.0493 - acc: 0.9841
Epoch 3/5
60000/60000 [==============================] - 29s 478us/sample - loss: 0.0349 - acc: 0.9887
Epoch 4/5
60000/60000 [==============================] - 29s 476us/sample - loss: 0.0274 - acc: 0.9915
Epoch 5/5
60000/60000 [==============================] - 29s 479us/sample - loss: 0.0209 - acc: 0.9934


In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 146us/sample - loss: 0.0274 - acc: 0.9921


In [7]:
print(test_acc)


0.9921
